In [0]:
select TradingDate, avg(DollarsPerMegawattHour) as Energy_spot_price
from datahub.wholesale.predispatchandfinalenergyprices
where TradingDate >= '2021-07-01'
and PriceTypeCode = 'F'
and PointOfConnectionCode = 'OTA2201'

In [0]:
with L1 as (
select TradingDate, GasVWAPDollarsPerGigajoule as `Daily volume weighted average gas price`, 
LAST_VALUE(CASE WHEN CarbonVWAPDollarsPerNZU <> 0 THEN CarbonVWAPDollarsPerNZU END, TRUE) OVER (ORDER BY TradingDate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
) AS `Daily volume weighted average carbon price`
from  datahub.wholesale.gasandcarbonpriceindices where TradingDate >= '2021-01-01'
), L2 as (
select TradingDate, avg(DollarsPerMegawattHour) as Energy_spot_price
from datahub.wholesale.predispatchandfinalenergyprices
where TradingDate >= '2021-07-01'
and PriceTypeCode = 'F'
and PointOfConnectionCode = 'OTA2201'
group by all
)
select L1.*, Energy_spot_price from L1 inner join L2 on L1.TradingDate = L2.TradingDate
order by L1.TradingDate


Databricks visualization. Run in Databricks to view.

In [0]:
select date'2021-07-01' as TradingDate, CarbonVWAPDollarsPerNZU
from datahub.wholesale.gasandcarbonpriceindices 
where TradingDate = (
select max(TradingDate) from datahub.wholesale.gasandcarbonpriceindices
where TradingDate <= date'2021-07-01' and CarbonVWAPDollarsPerNZU <> 0
)
union 
select TradingDate, CarbonVWAPDollarsPerNZU
from datahub.wholesale.gasandcarbonpriceindices
where TradingDate > date'2021-07-01'
order by Tradingdate